In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [2]:
%%time
land_specs_df = pd.read_csv('./Prepped Data/land_specs_baseline_20211116.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8549302, 16)
Wall time: 16.7 s


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자
0,서울특별시 종로구 청운동,일반,1,1,2013,1,대,15622.1,제1종일반주거지역,지정되지않음,연립,평지,사다리형,소로한면,2840000,2017-10-17
1,서울특별시 종로구 청운동,일반,1,1,2014,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,2933000,2017-10-17
2,서울특별시 종로구 청운동,일반,1,1,2015,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3053000,2017-10-17
3,서울특별시 종로구 청운동,일반,1,1,2016,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3234000,2017-10-17
4,서울특별시 종로구 청운동,일반,1,1,2017,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3488000,2018-05-23


In [4]:
land_specs_df = land_specs_df.sort_values(['법정동명', '대장구분명', '지번', '기준년도', '기준월']).reset_index(drop=True)
print(land_specs_df.shape)
land_specs_df.head()

(8549302, 16)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자
0,서울특별시 강남구 개포동,산,136-4,999999,2013,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,164000,2017-10-17
1,서울특별시 강남구 개포동,산,136-4,999999,2014,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,169000,2017-10-17
2,서울특별시 강남구 개포동,산,136-4,999999,2015,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,175000,2017-10-17
3,서울특별시 강남구 개포동,산,136-4,999999,2016,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,178500,2017-10-17
4,서울특별시 강남구 개포동,산,136-4,999999,2017,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,190000,2018-05-23


In [5]:
def find_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return '글자없음'
    else:
        return x

In [6]:
land_specs_df['글자'] = land_specs_df['지번'].apply(find_letters)

In [7]:
land_specs_df['글자'].value_counts()

글자없음    8544963
단          1738
D           411
종           303
C           208
근생          153
상업          110
편익           99
연녹           94
행당           91
주            81
DS           74
준주           73
B            58
공공           58
Ds           50
주유           48
유치           40
군            37
단독           34
자족           32
아            32
사회           27
공익           26
공동           22
고            19
자동           19
종교           19
초            18
S            16
어            16
CP           16
청사           15
커뮤           14
T            14
공            12
차고           10
문            10
체육           10
주차           10
I            10
도로           10
학            10
의료            9
근린            9
자원            9
폐             9
경             8
근             6
업C            6
DP            6
학교            6
화훼            6
주유P           5
상             5
경관            5
주I            5
가스Q           4
문화            4
보             4
단독R           4
도서            4
업무      

In [10]:
land_specs_df['대장구분명'].value_counts()

일반             8360481
산               152495
가지번              15241
블럭지번(지구)         11268
블럭지번              5322
기타지번              2949
블럭지번(지구_롯트)       1195
가지번(부번세분)          348
Name: 대장구분명, dtype: int64

In [11]:
land_specs_df['대장구분명'] = land_specs_df['대장구분명'].apply(lambda x: '산' if x == '산' else '일반')

In [12]:
land_specs_df['대장구분명'].value_counts()

일반    8396807
산      152495
Name: 대장구분명, dtype: int64

In [15]:
land_specs_df = land_specs_df.drop_duplicates(subset=['법정동명', '대장구분명', '지번', '기준년도'], keep='first').reset_index(drop=True)
print(land_specs_df.shape)
land_specs_df.head()

(8364612, 17)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,글자
0,서울특별시 강남구 개포동,산,136-4,999999,2013,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,164000,2017-10-17,글자없음
1,서울특별시 강남구 개포동,산,136-4,999999,2014,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,169000,2017-10-17,글자없음
2,서울특별시 강남구 개포동,산,136-4,999999,2015,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,175000,2017-10-17,글자없음
3,서울특별시 강남구 개포동,산,136-4,999999,2016,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,178500,2017-10-17,글자없음
4,서울특별시 강남구 개포동,산,136-4,999999,2017,1,임야,7017.0,개발제한구역,자연녹지지역,자연림,완경사,부정형,맹지,190000,2018-05-23,글자없음


In [8]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_coor_20211223.csv')
main_data_df = main_data_df.rename(columns={'글자':'대장구분명'})
print(main_data_df.shape)
main_data_df.head()

(5097504, 22)
Wall time: 13.1 s


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,대장구분명,지목,x좌표,y좌표
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,글자없음,대,204626.688971,442141.82001
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,글자없음,대,204626.688971,442141.82001
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,글자없음,대,204626.688971,442141.82001
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음,대,204626.688971,442141.82001
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,글자없음,대,204626.688971,442141.82001


In [9]:
main_data_df['대장구분명'].value_counts()

글자없음    5095189
산          2315
Name: 대장구분명, dtype: int64

In [13]:
main_data_df['대장구분명'] = main_data_df['대장구분명'].apply(lambda x: '산' if x == '산' else '일반')

In [14]:
main_data_df['대장구분명'].value_counts()

일반    5095189
산        2315
Name: 대장구분명, dtype: int64